In [1]:
import sys
import json
import pickle
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas, pd_writer
import getpass as gt
import pandas as pd
import numpy as np

from snowflake_conn import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [3]:
conn = get_connection()

### Step 1. Raw data fed into database ("staging")
- medical claim: `SANDBOX_FFS.TUVA_CLAIMS.MEDICAL_CLAIM` (or `SANDBOX_FFS.CORE.MEDICAL_CLAIM`?)
- eligibility: `SANDBOX_FFS.TUVA_CLAIMS.ELIGIBILITY` (or `SANDBOX_FFS.CORE.ELIGIBILITY`?)
- condition: `SANDBOX_FFS.CORE.CONDITION`
- Tuva seeds: start with `_value_set_`. In this notebook:
    - Adjustment rates: `seed_adjustment_rates`
    - HCPCS: `seed_hcpcs`
    - Demographic factors: `seed_demographic_factors`
    - Disease factors: `seed_disease_factors`
    - Disease hierarchy: `seed_disease_hierarchy`
    - Disabled interaction factors: `seed_disabled_interaction_factors`
    - Disease interaction factors: `seed_disease_interaction_factors`
    - Enrollment interaction factors: `seed_enrollment_interaction_factors`
    - ICD-10 mappings: `seed_icd_mappings`
    - HCC count factors: `seed_hcc_count_factors`

In [5]:
# medical claim
sql = '''SELECT COUNT(DISTINCT(patient_id)), COUNT(DISTINCT(claim_id)), COUNT(DISTINCT(claim_end_date)) FROM SANDBOX_FFS.TUVA_CLAIMS.MEDICAL_CLAIM'''
print(read_sql(sql))

sql = '''SELECT * FROM SANDBOX_FFS.TUVA_CLAIMS.MEDICAL_CLAIM LIMIT 1000;'''
tmp = read_sql(sql)
tmp.head(1)

   count(distinct(patient_id))  count(distinct(claim_id))  \
0                         8726                     555218   

   count(distinct(claim_end_date))  
0                             2928  


,claim_id,claim_line_number,claim_type,patient_id,member_id,claim_start_date,claim_end_date,claim_line_start_date,claim_line_end_date,admission_date,discharge_date,admit_source_code,admit_type_code,discharge_disposition_code,place_of_service_code,bill_type_code,ms_drg_code,apr_drg_code,revenue_center_code,service_unit_quantity,hcpcs_code,hcpcs_modifier_1,hcpcs_modifier_2,hcpcs_modifier_3,hcpcs_modifier_4,hcpcs_modifier_5,rendering_npi,billing_npi,facility_npi,paid_date,paid_amount,total_cost_amount,allowed_amount,charge_amount,diagnosis_code_type,diagnosis_code_1,diagnosis_code_2,diagnosis_code_3,diagnosis_code_4,diagnosis_code_5,diagnosis_code_6,diagnosis_code_7,diagnosis_code_8,diagnosis_code_9,diagnosis_code_10,diagnosis_code_11,diagnosis_code_12,diagnosis_code_13,diagnosis_code_14,diagnosis_code_15,diagnosis_code_16,diagnosis_code_17,diagnosis_code_18,diagnosis_code_19,diagnosis_code_20,diagnosis_code_21,diagnosis_code_22,diagnosis_code_23,diagnosis_code_24,diagnosis_code_25,diagnosis_poa_1,diagnosis_poa_2,diagnosis_poa_3,diagnosis_poa_4,diagnosis_poa_5,diagnosis_poa_6,diagnosis_poa_7,diagnosis_poa_8,diagnosis_poa_9,diagnosis_poa_10,diagnosis_poa_11,diagnosis_poa_12,diagnosis_poa_13,diagnosis_poa_14,diagnosis_poa_15,diagnosis_poa_16,diagnosis_poa_17,diagnosis_poa_18,diagnosis_poa_19,diagnosis_poa_20,diagnosis_poa_21,diagnosis_poa_22,diagnosis_poa_23,diagnosis_poa_24,diagnosis_poa_25,procedure_code_type,procedure_code_1,procedure_code_2,procedure_code_3,procedure_code_4,procedure_code_5,procedure_code_6,procedure_code_7,procedure_code_8,procedure_code_9,procedure_code_10,procedure_code_11,procedure_code_12,procedure_code_13,procedure_code_14,procedure_code_15,procedure_code_16,procedure_code_17,procedure_code_18,procedure_code_19,procedure_code_20,procedure_code_21,procedure_code_22,procedure_code_23,procedure_code_24,procedure_code_25,procedure_date_1,procedure_date_2,procedure_date_3,procedure_date_4,procedure_date_5,procedure_date_6,procedure_date_7,procedure_date_8,procedure_date_9,procedure_date_10,procedure_date_11,procedure_date_12,procedure_date_13,procedure_date_14,procedure_date_15,procedure_date_16,procedure_date_17,procedure_date_18,procedure_date_19,procedure_date_20,procedure_date_21,procedure_date_22,procedure_date_23,procedure_date_24,procedure_date_25,data_source
0,-10000930037915201571,1,professional,-10000010254618,-10000010254618,None,2015-09-28,None,2015-09-28,None,None,None,None,None,11,None,None,None,None,1,None,None,None,None,None,None,9999971093,1063654341,None,None,91.040000,91.040000,None,113.800000,icd-10-cm,R4689,E781,J329,E119,D649,E089,B965,E848,E849,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,medicare_lds


In [7]:
# eligibility
sql = '''SELECT COUNT(DISTINCT(patient_id)) FROM SANDBOX_FFS.TUVA_CLAIMS.ELIGIBILITY;'''
print(read_sql(sql))

sql = '''SELECT * FROM SANDBOX_FFS.TUVA_CLAIMS.ELIGIBILITY LIMIT 1000;'''
tmp = read_sql(sql)
tmp.head(1)

   count(distinct(patient_id))
0                         4883


,patient_id,member_id,gender,race,birth_date,death_date,death_flag,enrollment_start_date,enrollment_end_date,payer,payer_type,dual_status_code,medicare_status_code,first_name,last_name,address,city,state,zip_code,phone,data_source
0,-10000010255354,None,male,other race,1952-01-01,None,0,2017-01-01,2017-03-31,medicare,medicare,00,10,None,None,None,None,None,None,None,medicare_lds


In [9]:
# condition
sql = '''SELECT COUNT(DISTINCT(patient_id)) FROM SANDBOX_FFS.CORE.ELIGIBILITY;'''
print(read_sql(sql))
#
sql = '''SELECT * FROM SANDBOX_FFS.CORE.ELIGIBILITY LIMIT 1000;'''
tmp = read_sql(sql)
tmp.head(1)

   count(distinct(patient_id))
0                         4883


,patient_id,member_id,gender,race,birth_date,death_date,death_flag,enrollment_start_date,enrollment_end_date,payer,payer_type,original_reason_entitlement_code,dual_status_code,medicare_status_code,first_name,last_name,address,city,state,zip_code,phone,data_source,tuva_last_run
0,-10000010255354,None,male,other race,1952-01-01,None,0,2017-01-01,2017-03-31,medicare,medicare,0,00,10,None,None,None,None,None,None,None,medicare_lds,2023-10-02 16:50:57.649470+00:00


Seeds

In [10]:
# adjustment rates
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._VALUE_SET_ADJUSTMENT_RATES;'''
seed_adjustment_rates = read_sql(sql)

print(seed_adjustment_rates.shape)
seed_adjustment_rates.head(1).T

(6, 4)


,0
model_version,CMS-HCC-V24
payment_year,2019
normalization_factor,1.038
ma_coding_pattern_adjustment,0.059


In [11]:
# hcpcs
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._VALUE_SET_ADJUSTMENT_RATES;'''
seed_hcpcs = read_sql(sql)

print(seed_hcpcs.shape)
seed_hcpcs.head(1).T

(6, 4)


,0
model_version,CMS-HCC-V24
payment_year,2019
normalization_factor,1.038
ma_coding_pattern_adjustment,0.059


In [12]:
# demographic factors
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._VALUE_SET_DEMOGRAPHIC_FACTORS;'''
seed_demographic_factors = read_sql(sql)

print(seed_demographic_factors.shape)
seed_demographic_factors.head(1).T

(424, 11)


,0
model_version,CMS-HCC-V24
factor_type,Demographic
enrollment_status,Continuing
plan_segment,None
gender,Female
age_group,0-34
medicaid_status,No
dual_status,Non
orec,Aged
institutional_status,No


In [13]:
# disabled interaction factors
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._VALUE_SET_DISABLED_INTERACTION_FACTORS;'''
seed_disabled_interaction_factors = read_sql(sql)

print(seed_disabled_interaction_factors.shape)
seed_disabled_interaction_factors.head(1).T

(8, 8)


,0
model_version,CMS-HCC-V24
factor_type,Disabled Interactions
enrollment_status,Continuing
institutional_status,Yes
short_name,DISABLED_HCC85
description,Disabled & Congestive Heart Failure (HCC 85)
hcc_code,85
coefficient,0.279


In [14]:
# disease factors
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._VALUE_SET_DISEASE_FACTORS;'''
seed_disease_factors = read_sql(sql)

print(seed_disease_factors.shape)
seed_disease_factors.head(1).T

(580, 10)


,0
model_version,CMS-HCC-V24
factor_type,Disease
enrollment_status,Continuing
medicaid_status,No
dual_status,Non
orec,Aged
institutional_status,No
hcc_code,1
description,HIV/AIDS
coefficient,0.335


In [15]:
# disease hierarchy 
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._VALUE_SET_DISEASE_HIERARCHY;'''
seed_disease_hierarchy = read_sql(sql)

print(seed_disease_hierarchy.shape)
seed_disease_hierarchy.head(1).T

(72, 4)


,0
model_version,CMS-HCC-V24
hcc_code,8
description,Metastatic Cancer and Acute Leukemia
hccs_to_exclude,9


In [16]:
# disease interaction factors
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._VALUE_SET_DISEASE_INTERACTION_FACTORS;'''
seed_disease_interaction_factors = read_sql(sql)

print(seed_disease_interaction_factors.shape)
seed_disease_interaction_factors.head(1).T

(223, 12)


,0
model_version,CMS-HCC-V24
factor_type,Disease Interactions
enrollment_status,Continuing
medicaid_status,No
dual_status,Non
orec,Aged
institutional_status,No
short_name,HCC47_gCancer
description,Immune Disorders (HCC 47) & Cancer (HCCs 8-12)
hcc_code_1,47


In [17]:
# enrollment interaction factors
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._VALUE_SET_ENROLLMENT_INTERACTION_FACTORS;'''
seed_enrollment_interaction_factors = read_sql(sql)

print(seed_enrollment_interaction_factors.shape)
seed_enrollment_interaction_factors.head(1).T

(7, 10)


,0
model_version,CMS-HCC-V24
factor_type,Medicaid and Originally Disabled Interactions
gender,None
enrollment_status,Continuing
medicaid_status,None
dual_status,None
orec,None
institutional_status,Yes
description,Medicaid
coefficient,0.061


In [18]:
# icd-10 mappings
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._VALUE_SET_ICD_10_CM_MAPPINGS;'''
seed_icd_mappings = read_sql(sql)

print(seed_icd_mappings.shape)
seed_icd_mappings.head(1).T

(55368, 4)


,0
payment_year,2019
diagnosis_code,A0103
cms_hcc_v24,115
cms_hcc_v24_flag,Yes


In [19]:
# hcc count factors
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._VALUE_SET_PAYMENT_HCC_COUNT_FACTORS;'''
seed_hcc_count_factors = read_sql(sql)

print(seed_hcc_count_factors.shape)
seed_hcc_count_factors.head(1).T

(36, 10)


,0
model_version,CMS-HCC-V24
factor_type,Payment HCC Counts
enrollment_status,Continuing
medicaid_status,No
dual_status,Non
orec,Aged
institutional_status,No
payment_hcc_count,4
description,4 payment HCCs
coefficient,0.006


### Step 2. Determine member eligibility
- Find all healthcare enrollments for each patient
- Define the number of months each patient was covered during the collection year.
- Filter data to only include enrollments during the collection year.
- Define whether an enrollment is New or Continuing. CMS defines a new enrollee as a beneficiary with < 12 months of coverage PRIOR to the payment year.
- Combine eligibility info and enrollment status info for the most recent eligibility for each patient
- Add age group information (e.g., 0-34, 35-44, 85-89...). This is the same for continuing and new enrollees EXCEPT for ages 65-69. For Continuing patients, this is one group (65-69). For New patients, each of these ages is its own group (65, 66, 67, 68, 69).
- Add "medicaid status" information -- Yes if eligible for Medicare AND Medicaid, No if only eligible for Medicare (??).
- Add "dual status" information -- whether a patient is eligible for Full, Partial, or None of Medicare (or Medicaid?)
- Add "OREC" data (aged, disabled, ESRD, null)
**Tuva does not add OREC data currently and is only using a proxy for this information**
- Add institutional status
**Tuva does not have this logic added yet and everyone is listed as No**
- Add plan segment data
**Tuva does not include this, as the data is not available at the moment**

In [20]:
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._INT_MEMBERS;'''
member_eligibility = read_sql(sql)

print("member eligibility".upper())
print(member_eligibility.shape)
member_eligibility.head(3).T

MEMBER ELIGIBILITY
(4883, 14)


,0,1,2
patient_id,-10000010275925,-10000010267876,-10000010255283
enrollment_status,New,Continuing,New
gender,Female,Female,Male
age_group,66,0-34,66
medicaid_status,No,Yes,No
dual_status,Non,Full,Non
orec,Disabled,Disabled,None
institutional_status,No,No,No
enrollment_status_default,False,False,False
medicaid_dual_status_default,False,False,False


### Step 3. Capture demographic factors and coefficients
- Get patient info from Step #2 above
- Get HCC coefficient info from demographic seed file, which has different coeffiecients for combinations of gender, age group, medicaid status, OREC, etc
- Combine coefficient data and patient data for new enrollees
- Combine coefficient data and patient data for continuing enrollees -- same as new enrollees except also including Dual Status and Institutional Status for Continuing (not New)
- Combine coefficient data and patient data for "other" enrollees. **Tuva Note: The CMS-HCC model does not have factors for ESRD or null medicare status for these edge-cases, we default to 'Aged' and dual_status is Non or Partial.**
- Combine all enrollee types together

In [21]:
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._INT_DEMOGRAPHIC_FACTORS;'''
demographic_factors = read_sql(sql)

print("demographic factors".upper())
print(demographic_factors.shape)
demographic_factors.head(3).T

DEMOGRAPHIC FACTORS
(4883, 15)


,0,1,2
patient_id,-10000010275925,-10000010260739,-10000010284643
enrollment_status,New,New,New
gender,Female,Male,Male
age_group,66,66,66
medicaid_status,No,No,No
dual_status,Non,Non,Non
orec,Disabled,Aged,Aged
institutional_status,No,No,No
enrollment_status_default,False,False,False
medicaid_dual_status_default,False,False,False


### Step 4. Get enrollment interaction factors
#### *No patients meet this criteria*
- Get patient info from Step #3 above
- Get HCC coefficient info from demographic interaction seed file, which has coefficients for combinations of Aged and "Originally Disabled" (and also with Medicaid status, institutional status, etc) for CONTINUING enrollees
- Connect Patient ID with a coefficient corresponding to the "Originally disabled" interactions for non-institutional Continuing members >= 65
- Connect Patient ID with a coeficient corresponding to interactions of Medicaid and institutional members
- Combine patient interaction info (steps c and d above)

**Why Null -- No members meet either of these criteria:**
- (1) Originally disabled interactions for non-institutional (continuing) members >= 65 
- (2) Medicaid interactions for institutional members

In [22]:
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._INT_ENROLLMENT_INTERACTION_FACTORS;'''
enrollment_interaction_factors = read_sql(sql)

print("enrollment interaction factors".upper())
print(enrollment_interaction_factors.shape)
enrollment_interaction_factors.head(3).T

ENROLLMENT INTERACTION FACTORS
(0, 6)


""
patient_id
description
coefficient
model_version
payment_year
tuva_last_run


Originally disabled interactions for non-institutional members >= 65

In [23]:
# check filtering
tmp = demographic_factors.copy()
demo = tmp.loc[
    (tmp.institutional_status == 'No') & 
    (tmp.orec == 'Disabled') & 
    (tmp.age_group.isin(['65-69','70-74','75-79','80-84','85-89','90-94','>=95']))
]
print(demo.shape)
demo.head(1)

# these patients DO exist -- this is fine

(52, 15)


,patient_id,enrollment_status,gender,age_group,medicaid_status,dual_status,orec,institutional_status,enrollment_status_default,medicaid_dual_status_default,institutional_status_default,coefficient,model_version,payment_year,tuva_last_run
256,-10000010267401,New,Female,70-74,No,Non,Disabled,No,True,False,True,1.174,CMS-HCC-V24,2023,2023-10-02 16:50:57.649470+00:00


In [24]:
# checking joining cols
cols = ['gender','enrollment_status','medicaid_status','dual_status','institutional_status']
for c in cols:
    d = demo[c].unique().tolist()
    i = seed_enrollment_interaction_factors[c].unique().tolist()
    s = set(d) & set(i)
    if len(s) == 0:
        print(c)

# this is a problem

enrollment_status


In [25]:
# check filtering
tmp = demographic_factors.copy()
demo = tmp.loc[
    (tmp.institutional_status == 'Yes') & 
    (tmp.medicaid_status == 'Yes') 
]
print(demo.shape)
demo.head(1)

# this is another problem

(0, 15)


,patient_id,enrollment_status,gender,age_group,medicaid_status,dual_status,orec,institutional_status,enrollment_status_default,medicaid_dual_status_default,institutional_status_default,coefficient,model_version,payment_year,tuva_last_run


### Step 5. Determine eligible conditions
- Get raw medical claims data and raw conditions data (ICD 10 CM codes)
- Get seed data about HCPCS codes and whether they were covered during the payment year
- Select professional claims that took place during the collection year and are covered during the payment year
- Select inpatient claims that took place during the collection year and have bill types that refer to inpatient claims (bill types 11X and 41X)
- Select outpatient claims that took place during the collection year, covered in the payment year, and have bill types that refer to outpatient claims (bill types 12X, 13X, 43X, 71X, 73X, 76X, 77X, 85X)
- Combine all eligible claims (created in steps c-e above)
- Add ICD-10 CM codes to each eligible patient claim

In [26]:
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._INT_ELIGIBLE_CONDITIONS;'''
eligible_conditions = read_sql(sql)

print("eligible conditions".upper())
print(eligible_conditions.shape)
eligible_conditions.head(3).T

ELIGIBLE CONDITIONS
(94794, 5)


,0,1,2
patient_id,-10000010263581,-10000010263748,-10000010264140
condition_code,Z604,E669,Z608
model_version,CMS-HCC-V24,CMS-HCC-V24,CMS-HCC-V24
payment_year,2023,2023,2023
tuva_last_run,2023-10-02 16:50:57.649470+00:00,2023-10-02 16:50:57.649470+00:00,2023-10-02 16:50:57.649470+00:00


### Step 6. Map HCC codes to patient diagnosis (condition) codes
- Pull in data we created in #5 above
- Pull in seed data that maps eligible diagnosis codes to HCC codes
**Note: Tuva currently only supports CMS HCC v24**
- Add HCC codes to patient data

In [31]:
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._INT_HCC_MAPPING;'''
condition_codes = read_sql(sql)

print("hcc codes".upper())
print(condition_codes.shape)
condition_codes.head(3).T

HCC CODES
(16073, 6)


,0,1,2
patient_id,-10000010277095,-10000010260669,-10000010277139
condition_code,I213,R569,E1022
hcc_code,86,79,18
model_version,CMS-HCC-V24,CMS-HCC-V24,CMS-HCC-V24
payment_year,2023,2023,2023
tuva_last_run,2023-10-02 16:50:57.649470+00:00,2023-10-02 16:50:57.649470+00:00,2023-10-02 16:50:57.649470+00:00


### Step 7. Accound for groups of HCCs that have a disease hierarchy
 - *Background*: For some conditions (but not all), CMS has come up with a hierarchy of which conditions from the group is most (to least) severe. For a given disease category (e.g., Cancers), they say which HCC code is "highest / most severe" (e.g., HCC code 8, which corresponds to Metastatic Cancer and Acute Leukemia) and which HCC codes to exclude, if the patient has multiple conditions in that category (e.g., HCC codes 9, 10, 11, and 12, corresponding to Lung and Other Severe Cancers, Lymphoma and Other Cancers, Colorectal, Bladder, and Other Cancers, and HCC 12 (not sure)).
- a. Get data from #6 above
- b. Get seed data regarding disease hierarchy (HCC codes and description)
- c. Select HCCs that do NOT have a hierarchy
- d. Select HCCs that DO have a hierarchy for evaluation in steps 5-7
- e. Group by patient and HCC codes to account for multiple HCC combinations. Here, the minimum HCC is included (following CMS's severity logic)
- f. Select lower-level HCCs in the hierarchy
- g. Select top-level HCCs not included in previous steps
- h. Combine patients from steps e-g above

In [32]:
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._INT_HCC_HIERARCHY;'''
disease_hierarchy = read_sql(sql)

print("hcc groups (hierarchy)".upper())
print(disease_hierarchy.shape)
disease_hierarchy.head(3).T

HCC GROUPS (HIERARCHY)
(7275, 5)


,0,1,2
patient_id,-10000010262863,-10000010274996,-10000010261002
model_version,CMS-HCC-V24,CMS-HCC-V24,CMS-HCC-V24
payment_year,2023,2023,2023
hcc_code,79,79,2
tuva_last_run,2023-10-02 16:50:57.649470+00:00,2023-10-02 16:50:57.649470+00:00,2023-10-02 16:50:57.649470+00:00


### Step 8. Add disease coefficients to patient data
- a. Pull in demographic info from # 3 and HCC hierarchy info from # 7 
- b. Get seed data that gives coefficients for combinations of HCC codes and patient info (e.g., OREC, institutional status) for Continuing enrollees
- c. Combine demographic and HCC info for each patient
- d. Add disease-related coefficients to patient data

In [33]:
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._INT_DISEASE_FACTORS;'''
disease_coefficients = read_sql(sql)

print("disease coefficients".upper())
print(disease_coefficients.shape)
disease_coefficients.head(3).T

DISEASE COEFFICIENTS
(38, 7)


,0,1,2
patient_id,-10000010276621,-10000010262100,-10000010283286
hcc_code,19,18,18
hcc_description,Diabetes without Complication,Diabetes with Chronic Complications,Diabetes with Chronic Complications
coefficient,0.087,0.423,0.340
model_version,CMS-HCC-V24,CMS-HCC-V24,CMS-HCC-V24
payment_year,2023,2023,2023
tuva_last_run,2023-10-02 16:50:57.649470+00:00,2023-10-02 16:50:57.649470+00:00,2023-10-02 16:50:57.649470+00:00


### Step 9. Add disease interaction coefficients to patient data 
- a. Pull in demographic info from # 3 and HCC hierarchy info from # 7 
- b. Get seed data that gives coefficients for combinations of MULTIPLE HCC codes (e.g., Immune Disorders (HCC 47) and Cancer (HCCs 8-12)) and patient info (e.g., OREC, institutional status) for Continuing enrollees
- c. Combine demographic and HCC info for each patient
- d. Add disease interaction-related coefficients to patient data

In [34]:
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._INT_DISEASE_INTERACTION_FACTORS;'''
disease_interaction_coefficients = read_sql(sql)

print("disease interaction coefficients".upper())
print(disease_interaction_coefficients.shape)
disease_interaction_coefficients.head(3).T

DISEASE INTERACTION COEFFICIENTS
(0, 8)


""
patient_id
hcc_code_1
hcc_code_2
description
coefficient
model_version
payment_year
tuva_last_run


### Step 10. Add disabled interaction coefficients to patient data
- a. Pull in demographic info from # 3 and HCC hierarchy info from # 7 
- b. Get seed data that gives coefficients for HCC codes for disabled, Continuing enrollees (e.g., Disabled & Congestive Heart Failure, HCC 85)
- c. Combine demographic and HCC info for each patient
- d. Add disabled/HCC interaction-related coefficients to patient data

In [35]:
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._INT_DISABLED_INTERACTION_FACTORS;'''
disabled_interaction_coefficients = read_sql(sql)

print("disabled interaction coefficients".upper())
print(disabled_interaction_coefficients.shape)
disabled_interaction_coefficients.head(3).T

DISABLED INTERACTION COEFFICIENTS
(0, 6)


""
patient_id
description
coefficient
model_version
payment_year
tuva_last_run


### Step 11. Add coefficients for total number of "payment HCCs" to patient data
- a. Pull in demographic info from # 3 and HCC hierarchy info from # 7 
- b. Get seed data that gives coefficients for the total number of "payment HCCs" (4, 5, 6, 7, 8, 9, 10+ payment HCCs) and patient info (e.g., OREC, institutional status) for Continuing enrollees
- c. Count the number of HCC codes per patient. If patient has > 10 HCC codes, recode to "10+"
- d. Add coefficients for number of HCC codes to patient data

In [36]:
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._INT_HCC_COUNT_FACTORS;'''
hcc_coefficients = read_sql(sql)

print("hcc coefficients".upper())
print(hcc_coefficients.shape)
hcc_coefficients.head(3).T

HCC COEFFICIENTS
(0, 6)


""
patient_id
description
coefficient
model_version
payment_year
tuva_last_run


### Step 12. Combine all patient risk factors
- a. Combine demographic info from # 3, enrollment interaction info from # 4, disease factor info from # 8, disease interaction info from # 9, disabled interaction info from # 10, and total number of HCCs info from #11

In [37]:
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC.PATIENT_RISK_FACTORS;'''
patient_risk_factors = read_sql(sql)

print("patient risk factors".upper())
print(patient_risk_factors.shape)
patient_risk_factors.head(3).T

PATIENT RISK FACTORS
(4921, 9)


,0,1,2
patient_id,-10000010276621,-10000010262100,-10000010283286
enrollment_status_default,False,False,False
medicaid_dual_status_default,False,False,False
institutional_status_default,True,True,True
risk_factor_description,Diabetes without Complication (HCC 19),Diabetes with Chronic Complications (HCC 18),Diabetes with Chronic Complications (HCC 18)
coefficient,0.087,0.423,0.340
model_version,CMS-HCC-V24,CMS-HCC-V24,CMS-HCC-V24
payment_year,2023,2023,2023
tuva_last_run,2023-10-02 16:50:57.649470+00:00,2023-10-02 16:50:57.649470+00:00,2023-10-02 16:50:57.649470+00:00


In [38]:
patient_risk_factors['coefficient'].isna().sum()

0

### Step 13. Calculate patient risk scores
- a. Pull in seed data that has adjustment factors based on the payment year (to normalize costs year to year?)
- b. Pull in patient risk factor info from # 12
- c. Calculate the raw risk score for each patient by summing up all the coefficients for that payment year (e.g., Patient 123 has a different raw risk score for 2018, 2019, 2020...)
- d. Calculate the normalized risk score for each patient (for that payment year) by dividing the raw risk score by the normalization factor for that payment year
- e. Calculate the payment risk score for each patient (for that payment year) by multiplying the normalized risk score by (1-adjustment), where the adjustment is set by MA (?).
**Note: The adjustment is the same for each year in our data, so I'm not sure what it's doing.**

In [39]:
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC.PATIENT_RISK_SCORES;'''
patient_risk_scores = read_sql(sql)

print("patient risk scores".upper())
print(patient_risk_scores.shape)
patient_risk_scores.head(3).T

PATIENT RISK SCORES
(4883, 7)


,0,1,2
patient_id,-10000010260454,-10000010261056,-10000010272008
raw_risk_score,0.595,0.834,0.920
normalized_risk_score,0.528,0.740,0.816
payment_risk_score,0.497,0.696,0.768
model_version,CMS-HCC-V24,CMS-HCC-V24,CMS-HCC-V24
payment_year,2023,2023,2023
tuva_last_run,2023-10-02 16:50:57.649470+00:00,2023-10-02 16:50:57.649470+00:00,2023-10-02 16:50:57.649470+00:00


# Look at data

In [53]:
# members
sql = '''SELECT * FROM SANDBOX_FFS.CMS_HCC._INT_MEMBERS;'''
mem = read_sql(sql)
mem = mem.iloc[:,:-3].sort_values(['patient_id']).reset_index(drop=True)

print(mem.shape)
mem.head()

(4883, 11)


,patient_id,enrollment_status,gender,age_group,medicaid_status,dual_status,orec,institutional_status,enrollment_status_default,medicaid_dual_status_default,institutional_status_default
0,-10000010254647,New,Male,0-34,Yes,Full,Disabled,No,True,False,True
1,-10000010254653,New,Female,35-44,Yes,Full,Disabled,No,True,False,True
2,-10000010254655,New,Female,65,No,Non,Aged,No,True,False,True
3,-10000010254656,New,Female,0-34,Yes,Full,Disabled,No,True,False,True
4,-10000010254671,New,Female,80-84,Yes,Full,Aged,No,True,False,True


In [41]:
# hcc description
sql = '''SELECT DISTINCT hcc_code, description FROM SANDBOX_FFS.CMS_HCC._value_set_disease_factors;'''
des = read_sql(sql)

d = dict(zip(des.hcc_code, des.description))
des.head()

,hcc_code,description
0,1,HIV/AIDS
1,2,"Septicemia, Sepsis, Systemic Inflammatory Resp..."
2,6,Opportunistic Infections
3,8,Metastatic Cancer and Acute Leukemia
4,9,Lung and Other Severe Cancers


In [55]:
# hcc -- most severe diagnosis when there is >1 diagnosis in the same group
sql = '''SELECT patient_id, hcc_code FROM SANDBOX_CLAIMS.CMS_HCC._INT_HCC_HIERARCHY;'''
hcc = read_sql(sql).sort_values(['patient_id', 'hcc_code']).reset_index(drop=True)
hcc['description'] = hcc['hcc_code'].map(d)

print(hcc.shape)
hcc.head()

(297, 3)


,patient_id,hcc_code,description
0,10002,85,Congestive Heart Failure
1,10004,108,Vascular Disease
2,10010,111,Chronic Obstructive Pulmonary Disease
3,10010,85,Congestive Heart Failure
4,10010,96,Specified Heart Arrhythmias


In [43]:
# risk scores
sql = '''
SELECT 
    patient_id,
    raw_risk_score,
    normalized_risk_score
FROM SANDBOX_FFS.CMS_HCC.PATIENT_RISK_SCORES;
'''
risk = read_sql(sql).sort_values('patient_id').reset_index(drop=True)

print(risk.shape)
risk.head()

(4883, 3)


,patient_id,raw_risk_score,normalized_risk_score
0,-10000010254647,0.000,0.000
1,-10000010254653,0.000,0.000
2,-10000010254655,0.520,0.461
3,-10000010254656,0.000,0.000
4,-10000010254671,1.353,1.201


In [51]:
# cost
sql = '''SELECT * FROM SANDBOX_FFS.FINANCIAL_PMPM.PMPM_PREP;'''
pmpm = read_sql(sql)
pmpm = pmpm.iloc[:,:-1].sort_values('patient_id').reset_index(drop=True)

print(pmpm.shape)

print(len(pmpm[pmpm.total_paid > 0.]))

(244819, 52)
50199


,patient_id,year_month,inpatient_paid,outpatient_paid,office_visit_paid,ancillary_paid,pharmacy_paid,other_paid,acute_inpatient_paid,ambulance_paid,ambulatory_surgery_paid,dialysis_paid,durable_medical_equipment_paid,emergency_department_paid,home_health_paid,hospice_paid,inpatient_psychiatric_paid,inpatient_rehabilitation_paid,lab_paid,office_visit_paid_2,outpatient_hospital_or_clinic_paid,outpatient_psychiatric_paid,outpatient_rehabilitation_paid,skilled_nursing_paid,urgent_care_paid,inpatient_allowed,outpatient_allowed,office_visit_allowed,ancillary_allowed,pharmacy_allowed,other_allowed,acute_inpatient_allowed,ambulance_allowed,ambulatory_surgery_allowed,dialysis_allowed,durable_medical_equipment_allowed,emergency_department_allowed,home_health_allowed,hospice_allowed,inpatient_psychiatric_allowed,inpatient_rehabilitation_allowed,lab_allowed,office_visit_allowed_2,outpatient_hospital_or_clinic_allowed,outpatient_psychiatric_allowed,outpatient_rehabilitation_allowed,skilled_nursing_allowed,urgent_care_allowed,total_paid,medical_paid,total_allowed,medical_allowed
0,-10000010254647,201602,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-10000010254647,201705,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-10000010254647,201704,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-10000010254647,201611,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,-10000010254647,201610,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [48]:
# this currently is summing over multiple years
grouped = pmpm.groupby(by="patient_id")[['total_paid','medical_paid','total_allowed','medical_allowed']].sum().reset_index(drop=False)
grouped = grouped.sort_values('patient_id').reset_index(drop=True)

print(grouped.shape)
grouped.head()

(4883, 5)


,patient_id,total_paid,medical_paid,total_allowed,medical_allowed
0,-10000010254647,40987.670000,40987.670000,0.000000,0.000000
1,-10000010254653,19601959.450000,19601959.450000,0.000000,0.000000
2,-10000010254655,0.000000,0.000000,0.000000,0.000000
3,-10000010254656,38630018.330000,38630018.330000,0.000000,0.000000
4,-10000010254671,99641.640000,99641.640000,0.000000,0.000000


In [56]:
# merge
df = mem.merge(hcc, on='patient_id', how="outer").merge(risk, on='patient_id', how="outer").merge(grouped, on='patient_id', how="outer")

cols = ['patient_id','hcc_code','raw_risk_score','normalized_risk_score','total_paid','medical_paid','total_allowed','medical_allowed']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

print(df.shape)
df.head()

(5180, 19)


,patient_id,enrollment_status,gender,age_group,medicaid_status,dual_status,orec,institutional_status,enrollment_status_default,medicaid_dual_status_default,institutional_status_default,hcc_code,description,raw_risk_score,normalized_risk_score,total_paid,medical_paid,total_allowed,medical_allowed
0,-10000010254647,New,Male,0-34,Yes,Full,Disabled,No,True,False,True,NaN,NaN,0.000,0.000,40987.67,40987.67,0.0,0.0
1,-10000010254653,New,Female,35-44,Yes,Full,Disabled,No,True,False,True,NaN,NaN,0.000,0.000,19601959.45,19601959.45,0.0,0.0
2,-10000010254655,New,Female,65,No,Non,Aged,No,True,False,True,NaN,NaN,0.520,0.461,0.00,0.00,0.0,0.0
3,-10000010254656,New,Female,0-34,Yes,Full,Disabled,No,True,False,True,NaN,NaN,0.000,0.000,38630018.33,38630018.33,0.0,0.0
4,-10000010254671,New,Female,80-84,Yes,Full,Aged,No,True,False,True,NaN,NaN,1.353,1.201,99641.64,99641.64,0.0,0.0


Summary stats

In [60]:
# corr
tmp = df.loc[~df.hcc_code.isna(),['hcc_code','total_paid','total_allowed','normalized_risk_score']]
tmp[['total_paid','total_allowed','normalized_risk_score']].describe()
# df.loc[~df.hcc_code.isna(),['hcc_code','total_paid','total_allowed','normalized_risk_score']].corr().round(2)

,total_paid,total_allowed,normalized_risk_score
count,0.0,0.0,0.0
mean,NaN,NaN,NaN
std,NaN,NaN,NaN
min,NaN,NaN,NaN
25%,NaN,NaN,NaN
50%,NaN,NaN,NaN
75%,NaN,NaN,NaN
max,NaN,NaN,NaN


In [ ]:
# distribution of cost by quartiles (avg patient should cost $10k / yr)
df['quartile'] = (pd.qcut(df['total_paid'].values, 4, labels=[1,2,3,4]))
df.groupby('quartile')['total_paid'].agg(['min','mean','max']).round(2)

In [ ]:
# hccs by quartile
l = [
    'Congestive Heart Failure', 
    'Acute Myocardial Infarction',
    'Breast, Prostate, and Other Cancers and Tumors',
    'Acute Renal Failure',
    'Specified Heart Arrhythmias','Diabetes without Complication',
    'Rheumatoid Arthritis and Inflammatory Connective Tissue Disease',
    'Major Depressive, Bipolar, and Paranoid Disorders'
]

df.loc[df.description.isin(l)].groupby(['description','quartile']).size()

In [ ]:
# if had more patients
# split population into deciles by HCC (hard with 1000 patients)
# find the avg predicted cost in each decile (need prob coef that maps HCCs to predicted cost) / actual total cost

Admissions / Discharge

In [61]:
# encounter
sql = '''
SELECT 
    encounter_id, 
    patient_id, 
    encounter_start_date,
    encounter_end_date,
    encounter_type,
    admit_source_description,
    admit_type_description,
    discharge_disposition_description,
    paid_amount,
    charge_amount
FROM SANDBOX_FFS.CORE.ENCOUNTER;
'''
enc = read_sql(sql)
enc = enc.sort_values('patient_id').reset_index(drop=True)
enc[['patient_id','paid_amount','charge_amount']] = enc[['patient_id','paid_amount','charge_amount']].apply(pd.to_numeric, errors='coerce')

print(enc.shape)
enc.head()

(0, 10)


,encounter_id,patient_id,encounter_start_date,encounter_end_date,encounter_type,admit_source_description,admit_type_description,discharge_disposition_description,paid_amount,charge_amount


In [62]:
# discharge info
# for post acute patients (discharged from admission to somewhere else like skilled nursing). Makes things expensive. 

l = [
    'Discharged to home/self-care (routine charge)',
    'Discharged/transferred to an inpatient rehabilitation facility including distinct parts units of a hospital.',
    'Discharged/transferred to skilled nursing facility (SNF) with Medicare certification in anticipation of covered skilled care.',
]
enc.loc[enc['discharge_disposition_description'].isin(l),:].groupby('discharge_disposition_description')['paid_amount'].agg(['min','mean','max']).round(2)

,min,mean,max
discharge_disposition_description,,,


In [ ]:
enc.loc[enc['paid_amount']>130000.,:]

In [ ]:
# admission info
# look for expensive patients (do they have admissions)
enc['quartile'] = (pd.qcut(enc['paid_amount'].values, 4, labels=[1,2,3,4]))
enc.head()

res = pd.DataFrame(enc.groupby('admit_source_description')['quartile'].value_counts()).sort_values(['admit_source_description','quartile'])
res

In [ ]:
enc.groupby('patient_id')['encounter_id'].agg('count')

In [ ]:
pt = pd.DataFrame()
pt['patient_id'] = enc.patient_id.unique()
n = enc.groupby('patient_id')['encounter_id'].agg('count')
s = enc.groupby('patient_id').encounter_start_date.min()
e = enc.groupby('patient_id').encounter_start_date.max()
pt['num_encounters'] = pt['patient_id'].map(dict(zip(n.index, n.values)))
pt['first'] = pt['patient_id'].map(dict(zip(s.index, s.values)))
pt['last'] = pt['patient_id'].map(dict(zip(e.index, e.values)))
pt[['first','last']] = pt[['first','last']].astype('datetime64[ns]')
pt['diff'] = (pt['last'] - pt['first']).dt.days
# pt = pt[pt['diff'] > 365]
pt.num_encounters.describe()

In [ ]:
enc[enc.patient_id.isin([10252, 10573, 10779, 1149, 12506])]

HCC category to predicted cost

In [ ]:
# to do -- not enough data?

In [ ]:
close_connection()